In [5]:
#Run this in your machine before working with the model to install it:
!pip install pygsl

In [7]:
import nest
nest.set_verbosity('M_ERROR')

from pynestml.frontend.pynestml_frontend import generate_nest_target


generate_nest_target(input_path="models/adex_gamma_E.nestml",   # file containing the neuron model definition
                     suffix="_ml",                               # append to model name to avoid clash with existing model
                     module_name="m1module",                     # enumerate modules; name must end in "module"
                     target_path="tgt",                          # path for auxiliary files generated by NESTML
                     install_path=".")                           # location for generated module; must be "." on EBRAINS

nest.Install('m1module')


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.4
 Built: Jun 28 2023 05:25:40

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
AN

GeneratedCodeBuildException: Error occurred during cmake! More detailed error messages can be found in stderr.

In [2]:
import nest.raster_plot
import nest.voltage_trace
import numpy as np
import matplotlib.pyplot as plt
nest.ResetKernel()
nest.resolution = 0.1
%matplotlib inline
rt = nest.GetDefaults('adex_gamma_E_ml', 'receptor_types')

duration = 1000

# Define pop numbers
N_E = 160
N_I = 40

net = (2*N_E)+(2*N_I)

# Define Synaptic Weights
nmdaW = 1.0
ampaW = 5
gabaW = -3.34

# Set non-uniform initial membrane voltage
VmProbE = np.random.uniform(low=-65, high=-50, size=N_E)
VmProbI= np.random.uniform(low=-65, high=-50, size=N_I)

# Define syn params
stim_syn = {'weight': nest.random.lognormal(mean=0.9, std=0.3), 'receptor_type': rt['AMPA'], 'delay':1.5}  #nest.random.lognormal(mean=0.9, std=0.3)
noise_synE = {'weight': nest.random.lognormal(mean=0.2, std=0.05), 'receptor_type': rt['AMPA'], 'delay':1.5}
noise_synI = {'weight': nest.random.lognormal(mean=-0.2, std=0.05), 'receptor_type': rt['GABA'], 'delay':1.5}
EEsyn = nest.CollocatedSynapses({'weight': ampaW, 'receptor_type': rt['AMPA'], 'delay': 1.5},
                                {'weight': 0.4, 'receptor_type': rt['NMDA'], 'delay': 1.5})
EIsyn = nest.CollocatedSynapses({'weight': ampaW, 'receptor_type': rt['AMPA'], 'delay': 1.5},
                                {'weight': 0.5, 'receptor_type': rt['NMDA'], 'delay': 1.5})

localIsyn = {'weight': gabaW, 'receptor_type': rt['GABA'], 'delay': 1.5}
distIsyn = {'weight': gabaW, 'receptor_type': rt['GABA'], 'delay': 1.5}


# Define conn params
connEE = {'rule': 'pairwise_bernoulli', 'p': 0.65, 'allow_autapses': False}
connEI = {'rule': 'pairwise_bernoulli', 'p': 0.5}
connIE = {'rule': 'pairwise_bernoulli', 'p': 0.6}
connStim = {'rule': 'pairwise_bernoulli', 'p': 0.2}


# Create cells
E1 = nest.Create('adex_gamma_E_ml', N_E, params={'V_m': VmProbE})
E2 = nest.Create('adex_gamma_E_ml', N_E, params={'V_m': VmProbE})
I1 = nest.Create('adex_gamma_E_ml', N_I, params={'V_th':-47.5,'a': 0, 'b':0,'Delta_T': 0.5, 'V_m': VmProbI})
I2 = nest.Create('adex_gamma_E_ml', N_I, params={'V_th':-47.5,'a': 0, 'b':0,'Delta_T': 0.5, 'V_m': VmProbI})

# Create devices
pg = nest.Create("poisson_generator", {'rate':15000})
noise1 = nest.Create("poisson_generator", {'rate':1500})
noise2 = nest.Create("poisson_generator", {'rate':1500})
E1spk = nest.Create('spike_recorder')
I1spk = nest.Create('spike_recorder')
E2spk = nest.Create('spike_recorder')
I2spk = nest.Create('spike_recorder')
vM = nest.Create('voltmeter')

# Connect devices
nest.Connect(E1, E1spk)
nest.Connect(I1, I1spk)
nest.Connect(E2, E2spk)
nest.Connect(I2, I2spk)
nest.Connect(vM, E1)

# stim syns
nest.Connect(pg, E1, syn_spec=stim_syn, conn_spec=connStim)
nest.Connect(pg, I1, syn_spec=stim_syn, conn_spec=connStim)
# nest.Connect(noise1, E1, syn_spec=noise_synE)
# nest.Connect(noise1, I1, syn_spec=noise_synE)
# nest.Connect(noise1, E2, syn_spec=noise_synE)
# nest.Connect(noise1, I2, syn_spec=noise_synE)
# nest.Connect(noise1, E1, syn_spec=noise_synI)
# nest.Connect(noise1, I1, syn_spec=noise_synI)
# nest.Connect(noise1, E2, syn_spec=noise_synI)
# nest.Connect(noise1, I2, syn_spec=noise_synI)

# EE syns
nest.Connect(E1, E2, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.82,  'allow_autapses': False})
nest.Connect(E1, E1, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.25,  'allow_autapses': False})
nest.Connect(E2, E1, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.49,  'allow_autapses': False}) 
nest.Connect(E2, E2, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.335,  'allow_autapses': False}) 

# EI syns
nest.Connect(E1, I2, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.425,  'allow_autapses': False}) 
nest.Connect(E1, I1, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.325,  'allow_autapses': False}) 
nest.Connect(E2, I1, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.43,  'allow_autapses': False})#0.425
nest.Connect(E2, I2, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.365,  'allow_autapses': False}) #0.375

# IE syns
nest.Connect(I1, E1, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.42,  'allow_autapses': False})
nest.Connect(I1, E2, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.23,  'allow_autapses': False}) 
nest.Connect(I2, E1, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.42,  'allow_autapses': False}) 
nest.Connect(I2, E2, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.325,  'allow_autapses': False}) 

# II syns
# nest.Connect(I1, I2, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.15})
# nest.Connect(I2, I1, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.15})
nest.Connect(I2, I2, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.275,  'allow_autapses': False})
nest.Connect(I1, I1, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.275,  'allow_autapses': False})

NESTErrors.UnknownComponent: UnknownComponent in SLI function GetDefaults_l: /adex_gamma_E_ml is not a known component.

In [ ]:
nest.Simulate(duration)

In [ ]:
nest.raster_plot.from_device(E1spk)
nest.raster_plot.from_device(I1spk)
nest.raster_plot.from_device(E2spk)
nest.raster_plot.from_device(I2spk)

In [ ]:
spkct = len(E2spk.events['times'])+len(E1spk.events['times'])+len(I2spk.events['times'])+len(I1spk.events['times'])
rate_net = spkct / duration * 1000.0 / netSize